# VAMOS Interactive Tutorial

This notebook is a **hands-on, runnable tutorial** that walks you through VAMOS end-to-end.
Run each cell in order -- every section builds on the last.

**What you will learn:**

| Section | Skill |
|---------|-------|
| 1 | Check your installation |
| 2 | Run your first optimization (one line) |
| 3 | Visualize the Pareto front |
| 4 | Define a custom problem with `make_problem()` |
| 5 | Compare multiple algorithms |
| 6 | Add constraints |
| 7 | Tune parameters |
| 8 | Export results for papers |

> **Tip:** If a cell fails, read the error -- it usually means a dependency is missing.  
> Install everything with: `pip install -e ".[all]"`

---
## 1. Installation Check

Let's verify that VAMOS and its key dependencies are available.

In [ ]:
import vamos
import numpy as np

print(f"VAMOS location : {vamos.__file__}")
print(f"NumPy version  : {np.__version__}")

# Quick smoke test -- if this cell runs, you are ready!
from vamos import optimize, make_problem
print("\nAll imports OK.")

---
## 2. Your First Optimization

One function call is all you need.

We will solve **ZDT1** -- a classic 2-objective benchmark with 30 decision variables -- using **NSGA-II**.

In [ ]:
result = optimize(
    "zdt1",               # built-in benchmark (30 vars, 2 objectives)
    algorithm="nsgaii",    # the most popular MOEA
    max_evaluations=5000,  # stop after 5 000 function evals
    seed=42,               # reproducible
    verbose=True,          # print progress
)

print(f"\nFound {len(result)} non-dominated solutions.")
print(f"Objective space shape: {result.F.shape}")
print(f"Decision space shape : {result.X.shape}")

**What just happened?**

- `result.F` contains the objective values (each row = one Pareto-optimal solution).
- `result.X` contains the corresponding decision variable vectors.
- NSGA-II evolved a population and returned the best trade-off set.

---
## 3. Visualize the Pareto Front

A scatter plot of objective values shows the trade-off curve.

In [ ]:
import matplotlib.pyplot as plt

F = result.F

fig, ax = plt.subplots(figsize=(7, 5))
ax.scatter(F[:, 0], F[:, 1], s=20, alpha=0.8, edgecolors="k", linewidths=0.3)
ax.set_xlabel("f₀ (minimize)")
ax.set_ylabel("f₁ (minimize)")
ax.set_title("ZDT1 -- Pareto Front (NSGA-II, 5 000 evals)")
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("The curve shows the Pareto front: you cannot improve f₀ without worsening f₁.")

### Try it yourself

Change `"zdt1"` to `"zdt2"`, `"zdt3"`, or `"dtlz2"` in the cell above and re-run. Each problem has a different front shape!

---
## 4. Define Your Own Problem with `make_problem()`

You don't have to use built-in benchmarks. Define a custom problem by writing a plain Python function.

**Rules:**
- The function receives `x` (a 1-D array of decision variables).
- It returns a list of objective values (all minimized).
- You specify `n_var`, `n_obj`, and `bounds`.

In [ ]:
def my_objectives(x):
    """Two objectives: a simple convex Pareto front."""
    f0 = x[0]                                           # minimize first variable
    g  = 1.0 + 9.0 * sum(x[1:]) / (len(x) - 1)        # auxiliary
    f1 = g * (1.0 - (x[0] / g) ** 0.5)                 # minimize this too
    return [f0, f1]


problem = make_problem(
    my_objectives,
    n_var=5,                                            # 5 decision variables
    n_obj=2,                                            # 2 objectives
    bounds=[(0.0, 1.0)] * 5,                            # all vars in [0, 1]
    name="my_first_custom",
)

print(f"Problem: {problem.name}")
print(f"  n_var={problem.n_var}, n_obj={problem.n_obj}")
print(f"  bounds: {list(zip(problem.xl.tolist(), problem.xu.tolist()))[:3]} ...")

In [ ]:
# Solve the custom problem -- same one-liner as before
result_custom = optimize(
    problem,
    algorithm="nsgaii",
    max_evaluations=3000,
    seed=7,
    verbose=True,
)

fig, ax = plt.subplots(figsize=(7, 5))
ax.scatter(result_custom.F[:, 0], result_custom.F[:, 1], s=20, alpha=0.8, edgecolors="k", linewidths=0.3)
ax.set_xlabel("f₀")
ax.set_ylabel("f₁")
ax.set_title("Custom Problem -- Pareto Front")
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### Try it yourself

Edit `my_objectives` above to create a different trade-off. For example:

```python
def my_objectives(x):
    f0 = x[0] ** 2
    f1 = (x[0] - 2) ** 2
    return [f0, f1]
```

This is the **Schaffer N.1** problem -- a concave Pareto front.

---
## 5. Compare Multiple Algorithms

VAMOS supports many MOEAs. Let's compare three on the same problem.

In [ ]:
algos = ["nsgaii", "moead", "spea2"]
results = {}

for algo in algos:
    r = optimize("zdt1", algorithm=algo, max_evaluations=5000, seed=42)
    results[algo] = r
    print(f"{algo:8s}  ->  {len(r)} solutions,  f0 range [{r.F[:,0].min():.4f}, {r.F[:,0].max():.4f}]")

# Plot all fronts on the same axes
fig, ax = plt.subplots(figsize=(8, 5))
for algo, r in results.items():
    ax.scatter(r.F[:, 0], r.F[:, 1], s=15, alpha=0.7, label=algo)

ax.set_xlabel("f₀")
ax.set_ylabel("f₁")
ax.set_title("Algorithm Comparison on ZDT1")
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### Try it yourself

- Add `"smsemoa"` or `"nsgaiii"` to the `algos` list.
- Change `max_evaluations` to see how budget affects quality.
- Try a harder problem like `"dtlz2"` (3 objectives).

---
## 6. Constrained Optimization

`make_problem()` supports constraints too. A constraint function returns values where **<= 0 means feasible**.

In [ ]:
def objectives(x):
    return [x[0], x[1]]

def constraints(x):
    # Solutions must satisfy: x0 + x1 >= 1  (rewrite as: 1 - x0 - x1 <= 0)
    return [1.0 - x[0] - x[1]]

constrained = make_problem(
    objectives,
    n_var=2,
    n_obj=2,
    bounds=[(0, 2), (0, 2)],
    constraints=constraints,
    n_constraints=1,
    name="constrained_example",
)

rc = optimize(constrained, algorithm="nsgaii", max_evaluations=3000, seed=42)

fig, ax = plt.subplots(figsize=(7, 5))
ax.scatter(rc.F[:, 0], rc.F[:, 1], s=20, alpha=0.8, edgecolors="k", linewidths=0.3)

# Draw the constraint boundary
xs = np.linspace(0, 2, 100)
ax.plot(xs, 1 - xs, "r--", label="x₀ + x₁ = 1 (boundary)")
ax.fill_between(xs, 0, np.maximum(1 - xs, 0), alpha=0.08, color="red", label="infeasible")

ax.set_xlabel("f₀")
ax.set_ylabel("f₁")
ax.set_title("Constrained Problem -- feasible Pareto front")
ax.legend()
ax.set_xlim(-0.1, 2.1)
ax.set_ylim(-0.1, 2.1)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"All solutions satisfy x₀+x₁ >= 1: {all(rc.F[:,0] + rc.F[:,1] >= 0.99)}")

---
## 7. Tuning Parameters

The `optimize()` API exposes the knobs you actually need.

| Parameter | What it does | Typical range |
|-----------|-------------|---------------|
| `max_evaluations` | Total function evaluation budget | 1 000 -- 100 000 |
| `population_size` | Solutions per generation | 50 -- 200 |
| `seed` | Random seed or list of seeds | int or list |
| `engine` | Execution backend | `"numpy"`, `"jax"` |

Let's see the effect of population size.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4), sharey=True)

for ax, pop in zip(axes, [20, 50, 200]):
    r = optimize("zdt1", algorithm="nsgaii", max_evaluations=5000, population_size=pop, seed=42)
    ax.scatter(r.F[:, 0], r.F[:, 1], s=12, alpha=0.8)
    ax.set_title(f"pop_size = {pop}  ({len(r)} pts)")
    ax.set_xlabel("f₀")
    ax.grid(alpha=0.3)

axes[0].set_ylabel("f₁")
fig.suptitle("Effect of Population Size on ZDT1", fontsize=14)
plt.tight_layout()
plt.show()

### Try it yourself

- Increase `max_evaluations` to 20 000. Does the front improve?
- Try `engine="jax"` if you have JAX installed -- it can be significantly faster.
- Pass `seed=[0, 1, 2, 3, 4]` to run 5 independent seeds and get statistical results.

---
## 8. Export Results for Papers

VAMOS can generate publication-ready output.

In [ ]:
# Save Pareto front to CSV
import pandas as pd

df = pd.DataFrame(result.F, columns=[f"f{i}" for i in range(result.F.shape[1])])
df.to_csv("pareto_front.csv", index=False)
print("Saved pareto_front.csv")
df.head()

In [ ]:
# Generate a LaTeX table snippet
from vamos.ux.api import result_to_latex

latex = result_to_latex(result)
print(latex)

### Publication-ready figure

Matplotlib makes it easy to produce high-quality vector graphics.

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4))
ax.scatter(result.F[:, 0], result.F[:, 1], s=10, c="steelblue", edgecolors="navy", linewidths=0.3, alpha=0.9)
ax.set_xlabel(r"$f_1$", fontsize=12)
ax.set_ylabel(r"$f_2$", fontsize=12)
ax.set_title("ZDT1 Pareto Front (NSGA-II)", fontsize=13)
ax.grid(alpha=0.2)
fig.savefig("pareto_front.pdf", bbox_inches="tight", dpi=300)
fig.savefig("pareto_front.png", bbox_inches="tight", dpi=300)
plt.show()
print("Saved pareto_front.pdf and pareto_front.png")

---
## Cheat Sheet

| Task | Code |
|------|------|
| Solve a benchmark | `optimize("zdt1", algorithm="nsgaii", max_evaluations=5000)` |
| Custom problem | `make_problem(fn, n_var=5, n_obj=2, bounds=[(0,1)]*5)` |
| Constrained | `make_problem(fn, ..., constraints=g, n_constraints=1)` |
| Compare algorithms | loop over `["nsgaii", "moead", "spea2"]` |
| Multiple seeds | `optimize(..., seed=[0,1,2,3,4])` |
| CLI quickstart | `vamos quickstart` |
| Scaffold problem | `vamos create-problem` |
| Visual builder | `vamos studio` |
| Self-check | `vamos check` |

---

## Next Steps

1. **Explore more problems**: See `vamos.available_problem_names()` for the full list.
2. **Visual builder**: Run `vamos studio` to define problems with a GUI.
3. **Advanced config**: Check `notebooks/0_basic/04_advanced_configuration.ipynb`.
4. **Developer docs**: See `docs/dev/add_problem.md` to contribute new problems.

Happy optimizing!